> ***I.E. Paromtchik, C. Laugier. "Autonomous Parallel Parking of a Nonholonomic Vehicle." IEEE 1996.***

- Parallel Parking Maneuver
    - A parallel parking maneuver consists of N iteratively repeated low-speed motions(backwards-forwards) with the coordinated lateral and longitudinal control aimed at obtaining the lateral displacement of the vehicle.

- Model of a nonholonomic vehicle
![model of a nonholonomic vehicle](pic/modelofcar.png)

- Kinematic equation

$
\begin{cases}
\dot x = v cos\phi cos\theta\\
\dot y = v cos\phi sin\theta\\
\dot \theta = \frac{v}{L} sin\phi
\end{cases} \tag{1}
$

$
\begin{align}
\phi(t) = \phi_{max} k_\phi A(t), \quad 0 \leq t \leq T \tag{2}\\
v(t) = v_{max} k_v B(t), \quad 0 \leq t \leq T \tag{3}
\end{align}
$
$where \; k_\phi = \pm 1$ corresponds to the right(+1) or left(-1) parking bay relative to the vehicle's location, $k_v = \pm 1$ corresponds to the forward(+!) or backward(-1) motion

$
A(t) = 
\begin{cases}
1, &0 \leq t < t', \\
cos\frac{\pi (t - t')}{T^*}, &t' \leq t \leq T - t', \\
-1, &T - t' < t \leq T
\end{cases} \tag{4}
$

$B(t) = 0.5(1- cos\frac{4\pi t}{T}), \quad 0 \leq t \leq T \tag{5})$
$where \; t' = \frac{T - T^*}{2}, \; T^* < T$

- Cost function for collision

$|(x_T - x_0)cos\theta_0 + (y_T - y_0)sin\theta_0| < D_l \tag{6}$
$|(x_0 - x_T)sin\theta_0 + (y_T - y_0)cos\theta_0| < D_w\tag{7}$

- Algorithm flow
![Parallel Parking](pic/ParallelParking.png)

    1. Get available longitudinal and lateral distance$(D_l, D_w)$ data from the sensor data
    2. Search for maximal values T and $\phi_{max}$ by evaluating the (1) with (2), (3) so that constaints (6), (7) are still satisfied
    3. Get the next state point by T and $\phi_{max}$ calculated in the previous step 2
    4. If the “parked” location is reached, stop; else, go to step 1

- summary
    - Pros
        - Use a segment of fixed curve to generate a path
    - Cons
        - Smoothness around the point of the path is not considered
        - The success of the algorithm depends on the initial position of the vehicle

---

> ***Y. Guo. "Design and implementation of trajectory planning for auto-parking system." M.S. Thesis. Control Science and Engineering. Southwest Jiaotong University. 2017***

- Configuration Space: $q = (x, y, \theta)$
- Algorithm flow
![parking](pic/parkingmodel.png)
    1. The map environment is convexly divided to form a convex polygon obstacle edge
    2. Obtain a path consisting of a broken line segment at a safe distance from the edge of the obstacle(consider the car's heading $\theta$)
    3. Mark the ith point where the curvature is discontinuous as $O_i$
    4. Calculate the point $q_i$ that meets the constraint and minimize the arc length ${\stackrel{{\mbox{$\Large{\frown}$}}}{q_{i-1}q_i}}$ + ${\stackrel{{\mbox{$\Large{\frown}$}}}{q_iq_{i+1}}}$
    5. Repeat step 3 and 4 until the end position is reached

- summary
    - Path generation is mainly based on obstacles and the general processing idea is similar to the Reed-Shepp method, which makes a dis-continuous curvature
    - Algorithm's time complexity may be higher(The author did not give the response time of this system)
    - Only one curvature optimization process at the inflection point (increased two arcs), so can not be used in the narrow environment

---

> ***M. Hsieh. U. Oezguner. "A Parking Algorithm for an Autonomous Vehicle." IEEE. 2008***

![Vehicle Model with Ackerman Steering](pic/modelAckerman.png)

- Use the local coordinate system to get the final arc path
    - A final path is a circular arc with a radius larger than minimum turning radius of the car
    - The final paht is generated by two points, the origin of the local coordinate and the car's position
    - The final path is found when the car's yaw angle equals to the tangent angle of the circle
![Local Coordinate](pic/localCoordinate.png)

$$\theta_{steer} = \frac{\theta_l + \theta_r}{2}$$
$$R = \frac{L_{fr}}{tan\theta_{steer}}$$
$$R_{min} = R_{turn} - \frac{W_c}{2}$$
$$R_{max} =
\begin{cases}
Parking \; Forward: \; \sqrt{(R_{turn} + \frac{W_c}{2})^2 + L_{fr}^2}\\
Parking \; Backward: \sqrt{(R_{turn} + \frac{W_c}{2})^2 + (L_c - L_{fr})^2}
\end{cases}
$$
$$
\begin{align}
X_0 = \frac{W_p}{2}\\
if \; (R_{max}^2 - (R_{min} + (\frac{W_p}{2})^2)^2) \geq 0\\
Y_0 = \sqrt{R_{max}^2 - (R_{min} + (\frac{W_p}{2})^2)^2}\\
else \quad Y_0 = -\sqrt{\frac{W_cW_p}{2}-(\frac{W_c}{4})^2}
\end{align}
$$

where $(x_0, \ y_0)$ is the origin of the local coordinate

- Algorithm flow
    1. Pull forward and steer in the direction of $\theta_{steer}$ with the maximum steering angle to find a final path
    2. if hit an obstacle or a boundry, change the moving direction from forward/backward to backward/forward
    3. if the yaw angle equals to the tengent angle of the final path and the path radius is larger than minimum turning radius, following the final path. If hit any obstacle or a boundary during following, backs to step 2
    4. After the path following is finished, forward straight to the final position
- summary
    - An arc path has the advantage of small path following error and computation simplicity
    - Has been proven that the final path always exists and the path radius is always larger than the car's minimum turning radius
    - There is a small regin where a car cannot find a final path, but the car can just pull forward or backward with any steering angle for a short distance to get out of this region

---

> ***L. Han. Q. Do. S. Member. "Unified Path Planner for Parking an Autonomous Vehicle based on RRT." IEEE. 2011***

- the traditional methods divide the parking problem into parallel parking, perpendicular parking and echelon parking, in this paper a RRT based path planner is implemented for autonomous vehicle parking problem, which treats all the situations in a unified manner
- An approximation of the distance metrics in configuration space
![An Approximate Distance Metrics](pic/DistanceMetrics.png)

$$
D(P, P_0) = 
\begin{cases}
2\pi R, \; \qquad if \parallel{O_1 - P}\parallel < R \ or \parallel{O_2 - P}\parallel < R \\
\parallel{P_0 - P}\parallel, \quad otherwise
\end{cases}
$$
where R is the minimum turning radius of the vehicle, P is the sample point 

$$M(q, q_0) = \sqrt{D^2(P, P_0) + \alpha (\theta - \theta_0)}$$
where $\alpha$ is user defined factor to balance the importance of position difference and the angular difference

- Smoothing the path: using a modified Ramer-Douglas-peucker algorithm(remove the redundant points)
![A Path With Redundant Points](pic/RedundantPoints.png)

- summary
    - The sampling method consumes too much running time, that makes it unapplicable for real time applications
    - The generated paths are not smooth-enough
    - The approximation of the distance metric can also be migrated to other planning algorithms.

---

> P. Abbeel. D. Dolgov. A. Ng. S. Thrun. "Apprenticeship Learning for Motion Planning with Application to Parking Lot Navigation." IEEE. 2008

- Apprenticeship Learning
    - In artificial intelligence, apprenticeship learning(learning from demonstrtion) is the process of learniung by observing an expert. In this paper, the author uses this method to learn different human driving styles.
    - What is learned by this method is the weight of each coefficient(potential terms) in the cost function.
    - Algorithm flow(similar to the EM algorithm)
        1. Randomly pick a weight vector $w^{(0)}$, set $j = 0$.
        2. Solve the potential-field motion-planning problems for the current weight vector $w^{(j)}$, i.e. find  
        
        $$\boldsymbol{S}^{(i)} = argmin_\boldsymbol{s}\boldsymbol{\Phi}_{w^{(j)}}^{(i)}(\boldsymbol{S})$$
        $where \; \boldsymbol{S}^{(i)}$ is i-th path, $\boldsymbol{\Phi}_{w^{(j)}}^{(i)}(\boldsymbol{S}) = \sum_{k = 1}^{p}{w_k\phi_k(\boldsymbol{S})}$, $\{\phi_k(\bullet)\}_{k = 1}^p$ are the potential-field terms  
        
        3. Compute the cumulative values of the potentials:  
        $$\mu_k^{(j)} = \sum_{i = 1}^m{\phi_k(\boldsymbol{S^{(i)}})}$$  
        4. Find the next estimate for the weight vector $w^{(j + 1)}$ as the solution to the following convex optimization problem, if  ∥𝑤∥≤𝜖  for some desired accuracy  𝜖 , then exit and return  𝑥, {𝑤0,𝜇(0),...,𝑤𝑗,𝜇(𝑗)} ; otherwise, set  𝑗=𝑗+1 , set  𝑤𝑗=𝑤∥𝑤∥  and go to step 2.
        
        $$
\underset{w,x}{min} \ \parallel{w}\parallel_2^2\\
s.t. \mu = \sum_j{x_j\mu^{(j)}}; \ x \geq 0\\
\sum_j{x_j} = 1; \ w \geq 0\\
w \geq \mu - \mu_E
$$
- Potential function(cost so far)
    - length of trajectory driven forward: $w_{fwd}\sum_{i:i>1,\delta_i=0}{\parallel{X_i-X_{i-1}}\parallel}$, where $\delta$ is the direction of motion
    - length of trajectory drive in reverse: $w_{rev}\sum_{i:i>1,\delta_i=1}{\parallel{X_i-X_{i-1}}\parallel}$
    - number of times the direction of motion switches: $w_{sw}\sum_{i:\delta_i\neq \delta_{i - 1}}{1}$
    - length of trajectory driven off-road: $w_{road}\sum_{i:R(S_i)=0}{\parallel{X_i-X_{i-1}}\parallel}$, where $R(s) = 1$ if the car is on the road
    - an aggregate distance of the trajectory to the road-lane graph: $w_{lane}\sum_{i}{D(X_i, \theta_i, G)}$, where $G = {\langle}V, E{\rangle}$ is a directed graph
    - a measure of misalignment of trajectory and the principal directions of the parking lot:$w_{dir} = \sum_i{sin^2(2(\theta_i-\alpha_i))}$, where $\alpha_i = \alpha(argmin_E(D(E,X_i)))$ be the angle of the edge  𝐸  nearest to the trajectory point  𝑋𝑖
    - a measure of smoothness of the trajectory: $w_{curv}\sum_{i>1,i<|S|}{(\triangle{X_{i+1}}-\triangle{X_i})^2}$
- Driving style
    - “Nice”: tell the driver to drive in the right lane whenever safely possible.
    - “Sloppy”: tell the driver it is okay to deviate from the standard lanes，also tell the driver to only use forward driving.
    - “Backward”: allow the driver to drive backward, but only when it makes for a shorter path to the goal.
- Summary
    - The algorithm finds a set of weights such that both at training and test time the cumulative values of the cost functions are close to those obtained by the expert.
    - Since the hard constraints(collision avoidance, minimum turning radius of the vehicle) must be satisfied regardless of driving style, can fix the weights at large values and do not include them in learning.

---

## Autonomous Parking framework(Demo)
- System Dynamics
    - To simplify the model, assume that the vehicle's lateral velocity is zero.
    - $X_{k+1} = X_{k} + \Delta T*\widetilde{f}(X_k, U_k)$
    ![The model of 4WS Vehicle](pic/model_4WScar.jpg)
- General process
    1. Pre-process the input map to convert obstacles into bounding boxes
    2. Get an initial path using hybrid A* or lattice
    3. Solving stable and feasible Paths by using nonlinear solver(optimization-based approaches) based on restricted conditions
    4. Smoothing the velocity curve obtained from the path(not necessary)
- Expected file structure  

    |-- AutonomousParking
        |-- main.py(computes collision-free trajectories for parking)
        |-- config
            |-- __init__.py
            |-- config.py(Various default parameters, such as the length of the car, etc.)
        |-- utilities
            |-- __init__.py
            |-- preprocessed_map.py
            |-- hybrid_a_star.py(Method 1)
            |-- reeds_shepp.py(Use this function to get an analytical solution)
            |-- lattice.py(Method 2)
            |-- nonlinearSolver.py
            |-- collision_check.py
            |-- smoother.py
            |-- visualize_Path.py